In [6]:

import numpy as np

def get_dimensions(matrix):
    return [len(matrix), len(matrix[0])]

def find_determinant(matrix, excluded=1):
    dims = get_dimensions(matrix)
    if dims == [2, 2]:
        return excluded * ((matrix[0][0] * matrix[1][1]) - (matrix[0][1] * matrix[1][0]))
    det = 0
    for j in range(dims[1]):

        sub = [
            [matrix[i][k] for k in range(dims[1]) if k != j]
            for i in range(1, dims[0])
        ]
        coeff = matrix[0][j] * ((-1) ** j)
        det += coeff * find_determinant(sub, 1)
    return det

def polynomial_from_matrix(mat):

    dims = get_dimensions(mat)
    if dims == [2, 2]:

        a, minus_b = mat[0]
        c, minus_d = mat[1]

        return [a*minus_d - (minus_b)*c, -(a + minus_d), 1]

    from itertools import combinations

    return list(np.poly(np.linalg.eigvals(np.array([[row[i] for i in range(dims[1])] for row in mat], dtype=float))))

def find_eigenvalues_custom(A):

    coeffs = np.poly(np.array(A, dtype=float))
    return np.roots(coeffs)

def compare_with_numpy(A):
    custom = find_eigenvalues_custom(A)
    numpy_vals, _ = np.linalg.eig(np.array(A, dtype=float))
    return custom, numpy_vals

if __name__ == "__main__":
    A = [[6, 1, -1],
         [0, 7,  0],
         [3, -1, 2]]

    custom_vals, numpy_vals = compare_with_numpy(A)
    print("Custom eigenvalues:", custom_vals)
    print("NumPy eigenvalues: ", numpy_vals)


Custom eigenvalues: [7. 5. 3.]
NumPy eigenvalues:  [5. 3. 7.]
